# Technical Factor - Envelopes (ENV)

In [1]:
import dai
import pandas as pd

In [2]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [3]:
sql = """
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
),

data_factor AS (
    SELECT
        date,
        instrument,
        close,
        m_avg(close, 14) AS ENV_MIDDLE,
        ENV_MIDDLE * (1 + 0.05) AS ENV_UPPER,
        ENV_MIDDLE * (1 - 0.05) AS ENV_LOWER,
        (close - ENV_MIDDLE) / ENV_MIDDLE AS DIST,
    FROM data_base
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(close > ENV_MIDDLE AND m_lag(close, 1) <= m_lag(ENV_MIDDLE, 1), 1, 0) AS TRBY1,
        IF(close < ENV_MIDDLE AND m_lag(close, 1) >= m_lag(ENV_MIDDLE, 1), 1, 0) AS TRSL1,
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF(DIST > 0 AND (DIST - m_lag(DIST, 1)) > 0, 1, 0) AS MTBY1,
        IF(DIST < 0 AND (DIST - m_lag(DIST, 1)) < 0, 1, 0) AS MTSL1,
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(close < ENV_LOWER AND close > m_lag(close, 1), 1, 0) AS RVBY1,
        IF(close > ENV_UPPER AND close < m_lag(close, 1), 1, 0) AS RVSL1,
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(m_lag(close, 1) <= m_lag(ENV_UPPER, 1) AND close > ENV_UPPER, 1, 0) AS BKBY1,
        IF(m_lag(close, 1) >= m_lag(ENV_LOWER, 1) AND close < ENV_LOWER, 1, 0) AS BKSL1,
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        close,
        ENV_MIDDLE,
        ENV_UPPER,
        ENV_LOWER,
        DIST,
        TRBY1,
        TRSL1,
        MTBY1,
        MTSL1,
        RVBY1,
        RVSL1,
        BKBY1,
        BKSL1,
    FROM data_factor
    JOIN data_signal_trend USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date, instrument
"""

In [4]:
df = dai.query(sql, filters={"date":[sd,ed], "instrument":instrument_list}).df()
df

,date,instrument,close,ENV_MIDDLE,ENV_UPPER,ENV_LOWER,DIST,TRBY1,TRSL1,MTBY1,MTSL1,RVBY1,RVSL1,BKBY1,BKSL1
0,2025-01-21,600519.SH,11959.418608,11868.318235,12461.734147,11274.902323,0.007676,0,0,0,0,0,0,0,0
1,2025-01-22,600519.SH,11738.257136,11840.971249,12433.019812,11248.922687,-0.008674,0,1,0,1,0,0,0,0
2,2025-01-23,600519.SH,11754.548957,11822.352025,12413.469626,11231.234424,-0.005735,0,0,0,0,0,0,0,0
3,2025-01-24,600519.SH,11697.527583,11820.024622,12411.025853,11229.023391,-0.010364,0,0,0,1,0,0,0,0
4,2025-01-27,600519.SH,11689.300213,11816.993180,12407.842839,11226.143521,-0.010806,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,2025-12-25,600519.SH,11945.482630,11822.288759,12413.403197,11231.174321,0.010420,0,0,1,0,0,0,0,0
226,2025-12-26,600519.SH,11945.144750,11834.148346,12425.855764,11242.440929,0.009379,0,0,0,0,0,0,0,0
227,2025-12-29,600519.SH,11842.682738,11848.899124,12441.344080,11256.454168,-0.000525,0,1,0,1,0,0,0,0
228,2025-12-30,600519.SH,11738.953676,11855.178758,12447.937696,11262.419820,-0.009804,0,0,0,1,0,0,0,0
